# Phonons of a MgO chain (1D) 



### EXERCISE
- First run the following lines of code. This will create a MgO chain and calculate the phonons. It should not be modified, all the control options (**k** point and supercell creation) are in the subsequent cells. Even if the cell should not be modified, every line has been commented in order for the student to understand what the code is doing. The output of the first code cell is the phonon dispersion for the MgO chain. (If you cannot see it, run the cell again).
Then move to the next cell to animate the phonons you are interested in visualising. 
<br><br>
The MgO chain is only periodic along the $x$ direction. Systems with dimentsionality lower than three can still be simulated by using a 3D periodic cell with the inclusion of a vacuum region. For example, in the following exercise, the MgO chain is simulated through a cell whose length along the x direction is the optimised distance between atoms in the chain and has a 10 Angrstrom length along the other two directions. This means that the replicas of the chain will be far enough to ensure there is no interaction among them. 
<br>


Animate the phonons at the $\Gamma$ point (**k**=0.0) by using a 10 supercell. To animate the phonons change the settings below (**k** point and modes). NB: you have to change the mode in the name of the file you want to plot.



In [ ]:
# THIS CELL SHOULD NOT BE MODIFIED
# the imports first, always
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from ase.io.trajectory import Trajectory
from math import pi
from ase.spacegroup import crystal
from ase.calculators.gulp import GULP
from warnings import filterwarnings

#This is just to make the output look nice, you can ingore the line.
filterwarnings('ignore')

#Here we are defining a function called animate_phonons that will be used in the next code cell
#to display the motion of the atoms for the selected phonons.
def animate_phonons(atoms,label,k_point,supercell,modes):
    #read the phonon eigenvectors (the displacement of the atoms) in the eigen_out file
    with open('{}.eig'.format(label)) as o:
        eigen_out = o.readlines()
    #the k point coordinates were passed as a string, now we need to transform it into a np array
    k_point = np.array([float(x) for x in k_point.split()])
    
    n_cells = np.prod(supercell)

    #Calculation of the phase factor for the supercell
    ind = np.indices(supercell).reshape(3, -1)
    phase_f = np.exp(2.j * pi * np.dot(k_point, ind))
    phase_f= phase_f.repeat(len(atoms))

    #the mode indices were passed as a string, now we need to transform it into a list
    modes = [int(x) for x in modes.split()]
    #Another format modification in order to find the k point in the output
    k_point = [format(x, '.6f') for x in k_point]
    # repeat the unit cell in order to create the supercell
    atoms = atoms * np.array(supercell)
    #center the structure (this is for visualisation purposes)
    atoms.center()
    #now read the displacement at the selected k point for the six modes
    #and write that to a trajectory file (.trj), which is the one read by view
    for mode in modes:
        disp = np.zeros((N_atoms, 3), dtype=complex)
        for i,line in enumerate(eigen_out):
            m = re.match(r'K point at \s*{}\s*{}\s*{}'.format(k_point[0],k_point[1],k_point[2]), line)
            if m:
                for j in range(N_atoms):
                    n = (mode-1)*(N_atoms+2) + 3
                    eigenvec = eigen_out[i+n+j].split()
                    if len(eigenvec) == 3:
                        eigenvec = np.array([complex(float(eigenvec[i]),float(0.0)) for i in range(0,3)])    
                    elif len(eigenvec) == 6:
                        eigenvec = np.array([complex(float(eigenvec[i]),float(eigenvec[i+3])) for i in range(0,3)])
                    disp[j] =eigenvec 
        tot_disp = np.vstack(n_cells * [disp]) * phase_f[:, np.newaxis]
        tot_disp /= 10
        traj = Trajectory('{}_mode{}.trj'.format(label,mode), 'w')
        for x in np.linspace(0, 2 * pi, 30, endpoint=False):
            atoms.set_positions((atoms.get_positions() + np.exp(1.j * x) *
                                         tot_disp).real)
            traj.write(atoms)
        traj.close()

#create the structure and calculate the phonons
basis = [[0.0,0.0,0.0],[0.5,0.0,0.0]]
#this lattice parameter was optimised for the MgO chain
a = 3.47026
#in order to keep a three dimensional cell, a large (b=c=10Angstrom) empy space is added around the MgO chain.
#This is a standard procedure in computational materials science to simulate surfaces or polymers.
atoms = crystal('MgO',basis=basis,spacegroup=1,cellpar=[a, 10, 10, 90, 90, 90],primitive_cell=False) 
N_atoms = atoms.get_number_of_atoms()

#calculate the phonons of the MgO chain by using the primitive cell
label = 'mgo_chain'
#n of points where the phonons are calculated along the path specified in options.
n_k_ch = 100
options = ['shrink 100 1 1','dispersion 1 {}'.format(n_k_ch), '-1.0 0.0 0.0 to 0.0 0.0 0.0 to 1.0 0.0 0.0','output phon '+label, 'output eig '+label]

#run the calculation
calc = GULP(label=label ,keywords='conp phon eigenvectors', library='ionic.lib',options=options)
atoms.calc = calc
atoms.get_potential_energy()

outputs = [label+'.eig',label+'.disp',label+'.dens']
outputs = [label+'.disp',label+'.dens']
for file in outputs:
    if os.path.exists(file):
        os.remove(file)

The two cells below can be used to see the list of **k** points you can animate. <br>
The first one will work on Linux/Mac OS, the second one should be used in a Windows environment.<br>
If you already know the coordinates of the **k** point you want to animate you can skip this part and just input the coordinate in the cell below and run it. You only need to input the $k_{x}$ coordinate, the others are 0 by default because the structure is only periodic along the x direction.

In [ ]:
# USE THIS CELL IF YOU ARE USING A LINUX/MAC COMPUTER
! grep '0.000000  0.000000  Weight =' mgo_chain.got

In [ ]:
# USE THIS CELL IF YOU ARE USING A WINDOWS COMPUTER
! findstr "0.000000  0.000000  Weight =" mgo_chain.got

### Animate the phonons
Now you are ready to animate the phonons and display them through the ASE GUI:<br><br>
*NB: you can only animate one vibration at the time. To animate a new phonon you need to stop the cell below from running either by closing the GUI or by using the stop button at the top of the notebook. If you want to see more than one phonon at the same time you can open a new window from the GUI by using the New option in the File menu and selecting the file you want to display.*
- first select the **k** point you would like to animate. The **k** point coordinates are given, in general, as fractional coordinates of the first Brillouin zone (FBZ), which is the the reciprocal lattice equivalent of the primitive cell. Also, they usually range between -0.5 and 0.5 because all the other points of the reciprocal space can be mapped back into this region through a translation. This means that all the information of the crystal is contained into the FBZ.<br>
You only need to input the $k_x$ component of the **k** point you want to visualise, because the MgO chain is only periodic along the $x$ direction. However, since we are using a 3D periodic cell as it is explained above, the '0.0 0.0' indices along the other two directions must be added. This is what the central part of the script does;
- then select the right supercell to visualise the phonons at such **k** point. Also for this option only the number of repeated cells along $x$ must be specified. The '1 1' indices, meaning that the only one cell is used along the $y$ and $z$ directions is added in the script.
- In order to display the phonons you will have to select the mode you want to visualise, at the bottom of the cell in the line '! ase gui mgo_chain_mode1.trj 2> tmp'

In [ ]:
# Which k point would you like to visualise? Please make sure it is in the k point list
#of the calculation. If you are not sure run the cell above.
k_point = 0.25

#How many units do you want to use to visualise the phonons?
supercell = 1

k_point = '{}  0.000000  0.000000'.format(k_point)
supercell = [supercell] + [1,1]
modes = '1 2 3 4 5 6'

#the following line calls the function defined in the cell above and 
#saves the trajectory (.trj) files that can be used to display the phonons throught the GUI
animate_phonons(atoms,'mgo_chain',k_point,supercell,modes)

# Write the name of the file you want to animate
# You will have to modify the number of the mode you want to plot
! ase gui mgo_chain_mode1.trj 2> tmp


**Question:** <br>
1. Think about periodicity. <br>
    - For k_point = 1 and supercell = 1 can you visualize the phonon vibration? and for k_point= 0.25? why? <br>
    - Generalise the conclusions drawn from the points above for any k point. Given a k point, what is the supercell you need to use to be sure to recover the periodicity of the vibration?<br>